<a href="https://colab.research.google.com/github/nadnik13/sparse_grid/blob/main/GPU_Grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create GRID 1D

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import math
import sys
from numba import cuda

In [ ]:
fdim = 1
grid_level = 7
n_points = 100
grid_type = "full"

In [ ]:
@cuda.jit('float64(float64)', device=True)
def fun_gpu(x):
    return math.sin(10 * x - 0.1)

In [ ]:
@cuda.jit('float64(int32, int32)', device=True)
def xli_gpu(l, i):
    return i * (2. ** (-l))

In [ ]:
@cuda.jit('float64(float64)', device=True)
def phi_1_gpu(x):
    if -1 <= x <= 1:
        return 1 - abs(x)
    return 0
@cuda.jit('float64(float64)', device=True)
def phi_2_gpu(x):
    if -1 <= x <= 1:
        return - (x-1)*(x+1)
    return 0

@cuda.jit('float64(int32, int32, float64)', device=True)
def phi_li_gpu(l, i, x):
    if l == 0:
        return phi_1_gpu(x * 2. ** l - i)
    else:
        return phi_2_gpu(x * 2. ** l - i)

In [ ]:
def add_a_cpu(e, fvals):
    e['a'] = fvals[e['l']][e['i']]
#     print(f"add_a: i={e['i']} l={e['l']} e['i'] // 4={e['i'] % 4} e['i'] // 3={e['i'] % 3}")
    if e['l'] == 1:
        e['a'] += -0.5 * (fvals[e['l']][e['i']- 1] + fvals[e['l']][e['i'] + 1])
    elif (e['i'] % 4 == 1 and e['l'] > 1):
        e['a'] += -0.125 * (3*fvals[e['l']][e['i'] - 1] + 6*fvals[e['l']][e['i'] + 1] - fvals[e['l']][e['i'] + 3])
    elif (e['i'] % 4 == 3 and e['l'] > 1):
        e['a'] += -0.125 * (3*fvals[e['l']][e['i'] + 1] + 6*fvals[e['l']][e['i'] - 1] - fvals[e['l']][e['i'] - 3])
#     print(f"a = {e['a']}")
    # print(f"e['l']={e['l']} e['i']={e['i']} a_recursion={e['a']}")
    return e

0 0


{'a': -0.09983341664682815, 'i': 0, 'l': 0}

# Считаем значения функций на gpu

In [ ]:
@cuda.jit('void(float64[:, :])')
def func_gpu(func_xli):
    i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x # Отображение потока на индекс массива
    j = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y # Отображение потока на индекс массива
    if (i >= func_xli.shape[0]) or (j >= func_xli.shape[1]):
      return

    func_xli[i][j] = fun_gpu(xli_gpu(i, j))

In [ ]:
max_l = grid_level + 1
max_i = 2**grid_level + 1

n = (max_l, max_i)

d_points = cuda.device_array_like(np.zeros((max_l, max_i)))

tpb = (32, 32)  #blocksize или количество потоков на блок, стандартное значение = 32
bpg = (int(np.ceil((n[0])/tpb[0])), int(np.ceil((n[1])/tpb[1])) )  # блоков на грид

func_gpu[bpg, tpb](d_points) # вызов ядра

# Перенос вывода с устройства на хост
fvals = d_points.copy_to_host()

In [ ]:
2**grid_level + 1

129

# Generate grid

In [ ]:
def buildGrid_cpu(grid_level, fvals):
    grid=[]

    grid = [add_a_cpu({'l': 1, 'i': 1},fvals)]
    for e in grid:
      if e['l'] == grid_level: #or abs(e['a']) < 1e-3:
          continue
      grid.append(add_a_cpu({'l': e['l'] + 1, 'i': 2 * e['i'] - 1}, fvals))
      grid.append(add_a_cpu({'l': e['l'] + 1, 'i': 2 * e['i'] + 1}, fvals))

    grid.append(add_a_cpu({'l': 0, 'i': 0},fvals))
    grid.append(add_a_cpu({'l': 0, 'i': 1},fvals))
    return grid.copy()

In [ ]:
# buildGrid_cpu(3,points)

In [ ]:
def prepareGrid(grid, fdim):
  grid_l = []
  grid_i = []
  grid_a = []

  for i in grid:
    grid_l.append(i['l'])
    grid_i.append(i['i'])
    grid_a.append(i['a'])
  
  grid_l = np.array(grid_l)
  grid_i = np.array(grid_i)
  grid_a = np.array(grid_a)

  if fdim > 1:
    grid_l = grid_l.reshape(-1,fdim)
    grid_i = grid_i.reshape(-1,fdim)
    grid_a = grid_a.reshape(-1,fdim)
  return grid_l, grid_i, grid_a

In [ ]:
grid = buildGrid_cpu(grid_level, fvals)
grid_l, grid_i, grid_a = prepareGrid(grid,1)

In [ ]:
grid_l.shape, grid_i.shape, grid_a.shape

((129,), (129,), (129,))

In [ ]:
@cuda.jit('void(int32[:], int32[:], float64[:], float64[:, :], float64[:])')
def eval_gpu(grid_l, grid_i, grid_a, r, x):
    i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x # Отображение потока на индекс массива
    j = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y # Отображение потока на индекс массива
    if (i >= r.size):
      return
    r[i][j] = grid_a[j] * phi_li_gpu(grid_l[j], grid_i[j], x[i])

In [ ]:
from numba import cuda # Библиотека Nvidia для работы с GPU 
import numpy as np 

def run(n, x):
  # Подробности об устройстве
  # device = cuda.get_current_device()

  # Перенос с хоста на устройство
  d_grid_l = cuda.to_device(grid_l)  # Перенос данных в глобальную память GPU
  d_grid_i = cuda.to_device(grid_i)  # Перенос данных в глобальную память GPU
  d_grid_a = cuda.to_device(grid_a)  # Перенос данных в глобальную память GPU
  d_r = cuda.device_array_like(np.zeros(n))

  tpb = (32, 32)  #blocksize или количество потоков на блок, стандартное значение = 32
  bpg = (int(np.ceil((n[0])/tpb[0])), int(np.ceil((n[1])/tpb[1])) )  # блоков на грид


  eval_gpu[bpg, tpb](d_grid_l, d_grid_i, d_grid_a, d_r, x) # вызов ядра

  # Перенос вывода с устройства на хост
  r = d_r.copy_to_host()
  return r

x = np.linspace(0., 1., num=100)
n = [n_points, grid_a.size]
r = run(n, x)

In [ ]:
%timeit run(n, x)

100 loops, best of 5: 2.62 ms per loop


# 3d_GPU_Grid.ipynb

In [ ]:
grid_levels = [1,2,3,5,7]
grid_level = 3
basis_dimensions = [1,2,4] # basis dimension
basis_dimension = 2 # basis dimension
gridTypes = ['full', 'sparse']
gridType = 'full'
n_points = 10
func_dimension = 3  # function dimension

In [ ]:
@cuda.jit('float64(float64[:])', device=True)
def fun_gpu3(x):
    return math.sin(5 * x[0]) + math.cos(6 * x[1]) + 0.01 * math.sin(10 * x[0] * x[1])

def fun_cpu3(x):
    return math.sin(5 * x[0]) + math.cos(6 * x[1]) + 0.01 * math.sin(10 * x[0] * x[1])

In [ ]:
@cuda.jit('float64(int32, int32)', device=True)
def xli_gpu(l, i):
    return i * (2. ** (-l))

In [ ]:
@cuda.jit('float64(float64)', device=True)
def phi_1(x):
    if -1 <= x <= 1:
        return 1 - abs(x)
    return 0

@cuda.jit('float64(float64)', device=True)
def phi_2(x):
    if -1 <= x <= 1:
        return - (x-1)*(x+1)
    return 0

@cuda.jit('float64(float64)', device=True)
def phi_4_1(x):
    if -1 <= x <= 3:
        return - 1/6 * (x+1)*(x-1)*(x-2)*(x-3)
    return 0

@cuda.jit('float64(float64)', device=True)
def phi_4_2(x):
    if -3 <= x <= 1:
        return - 1/6 * (x+3)*(x+2)*(x+1)*(x-1)
    return 0

@cuda.jit('float64(int32, int32, float64, int32)', device=True)
def phi_li_1(l, i, x, db):
    if db == 1:
        return phi_1(x * 2 ** l - i)
    return 0

@cuda.jit('float64(int32, int32, float64, int32)', device=True)
def phi_li_2(l, i, x, db):
        if l == 0:
            return phi_1(x * 2 ** l - i)
        else:
            return phi_2(x * 2 ** l - i)

@cuda.jit('float64(int32, int32, float64, int32)', device=True)
def phi_li_4(l, i, x, db):
    if l == 0:
        return phi_1(x * 2 ** l - i)
    elif l == 1:
        return phi_2(x * 2 ** l - i)
    elif i % 4 == 1:
        return phi_4_1(x * 2 ** l - i)
    elif i % 4 == 3:
        return phi_4_2(x * 2 ** l - i)
    return 0

@cuda.jit('float64(int32, int32, float64, int32)', device=True)
def phi_li_gpu3(l, i, x, db):
  if db == 1:
    return phi_li_1(l, i, x, db)
  if db == 2:
    return phi_li_2(l, i, x, db)
  if db == 4:
    return phi_li_4(l, i, x, db)
  return 0

# Считаем значения функций на gpu

In [ ]:
@cuda.jit('void(float64[:, :, :])')
def func_gpu(func_xli):
    i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x # Отображение потока на индекс массива
    j = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y # Отображение потока на индекс массива
    if (i >= func_xli.shape[0]) or (j >= func_xli.shape[1]):
      return
    for z in range(func_xli.shape[2]):
      x = [xli_gpu(i, j), ]

    func_xli[i][j][z] = fun_gpu3(xli_gpu(i, j))

In [ ]:
def add_a_cpu3(e, bdim, fdim, fval):
    if bdim == 1:
        e['a'] =  alik_1d(e['l'], e['i'], 0, fdim)
    if bdim == 2:
        e['a'] =  alik_2d(e['l'], e['i'], 0, fdim)
    if bdim == 4:
        e['a'] =  alik_4d(e['l'], e['i'], 0, fdim)
    return e

def alik_1d(l, i, k, fdim):
    if k == fdim:
        x = [xli_cpu(l[j], i[j]) for j in range(fdim)]
        return fun_cpu3(x)
    else:
        main_val = alik_1d(l,i,k+1, fdim)

        if (i[k] > 0 ) and l[k] > 0:  
            i_prev_1 = [val-1 if idx == k else val for idx, val in enumerate(i)]
            i_next_1 = [val+1 if idx == k else val for idx, val in enumerate(i)]
            return main_val - 0.5*(alik_1d(l,i_prev_1,k+1, fdim) + alik_1d(l,i_next_1,k+1, fdim))
        else:
            return main_val
        
def alik_2d(l, i, k, fdim):
    if k == fdim:
        x = [xli_cpu(l[j], i[j]) for j in range(fdim)]
        return fun_cpu3(x)
    else:
        main_val = alik_2d(l,i,k+1,fdim)

        if l[k] == 1 and i[k] > 0:
            i_prev_1 =  [val-1 if idx == k else val for idx, val in enumerate(i)]
            i_next_1 =  [val+1 if idx == k else val for idx, val in enumerate(i)]
            return main_val - 0.5 * (alik_2d(l,i_prev_1,k+1, fdim) + alik_2d(l,i_next_1,k+1, fdim))
        
        if l[k] > 1:
            i_prev_1 = [val-1 if idx == k else val for idx, val in enumerate(i)]
            i_next_1 = [val+1 if idx == k else val for idx, val in enumerate(i)]

            prev_1_val = alik_2d(l,i_prev_1,k+1, fdim)
            next_1_val = alik_2d(l,i_next_1,k+1, fdim)
            
            if (i[k] % 4 == 1):
                i_next_3 = [val+3 if idx == k else val for idx, val in enumerate(i)]
                next_3_val = alik_2d(l,i_next_3,k+1, fdim)
                return main_val - 0.125*(3*prev_1_val+6*next_1_val-next_3_val)
            
            if (i[k] % 4 == 3):
                i_prev_3 = [val-3 if idx == k else val for idx, val in enumerate(i)]
                prev_3_val = alik_2d(l,i_prev_3,k+1, fdim)
                return main_val - 0.125*(6*prev_1_val+3*next_1_val-prev_3_val)
        return main_val

def alik_4d(l, i, k, fdim):

    if k == fdim:
        x = [xli_cpu(l[j], i[j]) for j in range(fdim)]
        return fun_cpu3(x)
    else:
        main_val = alik_4d(l,i,k+1,fdim)
        i_prev_1 = [val-1 if idx == k else val for idx, val in enumerate(i)]
        i_next_1 = [val+1 if idx == k else val for idx, val in enumerate(i)]
        
        i_prev_3 = [val-3 if idx == k else val for idx, val in enumerate(i)]
        i_next_3 = [val+3 if idx == k else val for idx, val in enumerate(i)]
        
        i_prev_5 = [val-5 if idx == k else val for idx, val in enumerate(i)]
        i_next_5 = [val+5 if idx == k else val for idx, val in enumerate(i)]
        
        i_prev_7 = [val-7 if idx == k else val for idx, val in enumerate(i)]
        i_next_7 = [val+7 if idx == k else val for idx, val in enumerate(i)]

        if l[k] == 1 and i[k] > 0 :
            return main_val - 0.5 * ( alik_4d(l,i_prev_1,k+1, fdim) + alik_4d(l,i_next_1,k+1, fdim) )
        
        if l[k] == 2:
            prev_1_val = alik_4d(l,i_prev_1,k+1, fdim)
            next_1_val = alik_4d(l,i_next_1,k+1, fdim)
            
            if (i[k] % 4 == 1):
                next_3_val = alik_4d(l,i_next_3,k+1, fdim)
                return main_val - 0.125 * (3*prev_1_val + 6*next_1_val - next_3_val)
            
            if (i[k] % 4 == 3):
                prev_3_val = alik_4d(l,i_prev_3,k+1, fdim)
                return main_val - 0.125 * (6*prev_1_val + 3*next_1_val - prev_3_val)
        
        if l[k] > 2: 
            
            prev_1_val = alik_4d(l,i_prev_1,k+1, fdim)
            next_1_val = alik_4d(l,i_next_1,k+1, fdim)
            prev_3_val = alik_4d(l,i_prev_3,k+1, fdim)
            next_3_val = alik_4d(l,i_next_3,k+1, fdim)
            
            if (i[k] % 8 == 1):
                next_5_val = alik_4d(l,i_next_5,k+1, fdim)
                next_7_val = alik_4d(l,i_next_7,k+1, fdim)
                return main_val - 1/128 * (35*prev_1_val + 140*next_1_val - 70*next_3_val + 28*next_5_val - 5*next_7_val)
            
            elif (i[k] % 8 == 3):
                next_5_val = alik_4d(l,i_next_5,k+1, fdim)
                return main_val - 1/128 * (-5*prev_3_val + 60*prev_1_val + 90*next_1_val - 20*next_3_val + 3*next_5_val)
            
            elif (i[k] % 8 == 5):
                prev_5_val = alik_4d(l,i_prev_5,k+1, fdim)
                return main_val - 1/128 * (3*prev_5_val - 20*prev_3_val + 90*prev_1_val + 60*next_1_val - 5*next_3_val)
            
            elif (i[k] % 8 == 7):
                prev_5_val = alik_4d(l,i_prev_5,k+1, fdim)
                prev_7_val = alik_4d(l,i_prev_7,k+1, fdim)
                return main_val - 1/128 * (-5*prev_7_val + 28*prev_5_val - 70*prev_3_val + 140*prev_1_val + 35*next_1_val)
            
        return main_val